In [13]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Mark Viti \& Marcus Lisman

## Problem 1

### 1
**Problem:**
For $Y = X\beta + U$ where we observe $Y, \tilde{X}, \u{X}$ such that $\tilde{X} = X + V^1$ and $\u{X} = X + V^2$. Moreover, we have that for $j = {1, 2}$, $\mathbb{E}\left[V^j\right] = 0, \mathbb{E}[XV^j] = 0, \mathbb{E}[V^1V^2] = 0$, $\mathbb{E}\left[UV^j\right] = 0$, $\mathbb{E}\left[V^1V^2\right]$. Verify the relevance and exogeneity of $\u{X}$ as an instrument for $\tilde{X}$.

**Solution:**
Recall the definitions of relevance and exogeneity:
- Relevance: $\mathbb{E}\left[\u{X}\tilde{X}\right] \neq 0$
- Exogeneity: $\mathbb{E}\left[\u{X}U\right] = 0$

First, we verify relevance:
$\begin{align}
\mathbb{E}\left[\u{X}\tilde{X}\right] &= \mathbb{E}\left[\left(X + V^2\right)\left(X + V^1\right)\right] \\
&= \mathbb{E}\left[X^2 + XV^1 + XV^2 + V^1V^2\right] \\
&= \mathbb{E}\left[X^2\right] + \mathbb{E}\left[XV^1\right] + \mathbb{E}\left[XV^2\right] + \mathbb{E}\left[V^1V^2\right] \\
&= \mathbb{E}\left[X^2\right] + 0 + 0 + 0 \\
&= \mathbb{E}\left[X^2\right] \neq 0
\end{align}$
Now, someone asked on Ed if we can assume that $var{X} > 0$, which we can. This it is clear that $\mathbb{E}\left[\u{X}\tilde{X}\right] \neq 0$.

Next, we verify exogeneity:
$\begin{align}
\mathbb{E}\left[\u{X}U\right] &= \mathbb{E}\left[\left(X + V^2\right)U\right] \\
&= \mathbb{E}\left[XU + V^2U\right] \\
&= \mathbb{E}\left[XU\right] + \mathbb{E}\left[V^2U\right] \\
&= \mathbb{E}\left[XU\right] + 0 \\
&= 0
\end{align}$
Thus, $\u{X}$ is both relevant and exogenous as an instrument for $\tilde{X}$.
QED

### 2
**Problem:**
We have the following supply and demand equations:

$Q^s(P) = \alpha^s + P\beta^s + Z_\gamma + U^s$

$Q^d(P) = \alpha^d + P\beta^d + U^d$
- - - 
**(A)**: What is the expected sign of $\beta^s$, $\beta^d$, and $\beta^d - \beta^s$?

The expected sign of $\beta^s$ is positive, as the quantity supplied should increase with price. The expected sign of $\beta^d$ is negative, as the quantity demanded should decrease with price. Thus, the expected sign of $\beta^d - \beta^s$ is negative.

- - - 

**(B)**: Derive the equilibrium price. 

At equilibrium, $Q^s(P) = Q^d(P)$. Thus, we have:
$\begin{align}
\alpha^s + P\beta^s + Z_\gamma + U^s &= \alpha^d + P\beta^d + U^d \\
P\beta^s - P\beta^d &= \alpha^d - \alpha^s + U^s - U^d - Z_\gamma \\
P(\beta^s - \beta^d) &= \alpha^d - \alpha^s + U^s - U^d - Z_\gamma \\
P &= \frac{\alpha^d - \alpha^s + U^s - U^d - Z_\gamma}{\beta^s - \beta^d}
\end{align}$
We are going to call this price $P^*$.

- - - 

**(C)**: Show that the equilibrium price is endogenous in the model for demand.

We have that $P^* = \frac{\alpha^d - \alpha^s + U^s - U^d - Z_\gamma}{\beta^s - \beta^d}$. We can see that $P^*$ is endogenous in the model for demand because $U^d$ is a random variable that is not observed. Thus, $P^*$ is a function of $U^d$ and is endogenous in the model for demand.

- - -

**(D)**: Show that $Z$ is relevant and exogenous as an instrument for $P$ in the demand model. Assume that $\gamma \neq 0, \mathbb{E}\left[ZU^s\right] = \mathbb{E}\left[ZU^d\right] = \mathbb{E}\left[U^sU^d\right] = 0$ and $var(Z) > 0$.

To do so, we want to show that $\mathbb{E}\left[ZP\right] \neq 0$ and $\mathbb{E}\left[ZU^d\right] = 0$. Note that we are assuming that $\mathbb{E}\left[ZU^s\right] = 0$ and $\mathbb{E}\left[ZU^d\right] = 0$, so we already have exogeneity.

Thus, we need to show relevance. 
$\begin{align}
\mathbb{E}\left[ZP\right] &= \mathbb{E}\left[Z\frac{\alpha^d - \alpha^s + U^s - U^d - Z_\gamma}{\beta^s - \beta^d}\right] \\
&= \frac{\alpha^d - \alpha^s + \mathbb{E}\left[ZU^s\right] - \mathbb{E}\left[ZU^d\right] - \mathbb{E}\left[Z^2\gamma\right]}{\beta^s - \beta^d} \\
&= \frac{\alpha^d - \alpha^s - \mathbb{E}\left[Z^2\gamma\right]}{\beta^s - \beta^d} \neq 0
\end{align}$
Thus, $Z$ is relevant and exogenous as an instrument for $P$ in the demand model.

- - -

**(E)**: Is $Z$ relevant as an instrument for $P$ in the supply model?

No. $Z$ is in the model itself, so it is not relevant as an instrument for $P$ in the supply model.


## Problem 2

### 1

In [6]:
def generate_data(n: int, b1: float, b2: float, b3: float, b4: float, gamma1: float, gamma2: float):
    x2 = np.random.normal(0, 1, n)
    v = np.random.normal(0, 0.1, n)
    x1 = x2 + x2**2*gamma1 + x2**5*gamma2 + v
    u = np.random.normal(0, 1, n)
    y = b1 * x1 + b2 * x2 + x2**2*b3 + np.sin(x2)*b4 + u

    return (y, x1, x2)

### 2

### 3